In [1]:
import xgboost as xgb 
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

disease_list = df.SUBCLASS.unique()
df_target = df['SUBCLASS']

data = df.drop(columns=['ID', 'SUBCLASS'])
df_test.drop(columns=['ID'], inplace=True)
data

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [ ]:
df_ = data.copy()
df_['SUBCLASS'] = df.loc[:, df.columns[1:2]]
for d in disease_list:
    count_df = df_[df_.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum)
    count_df = count_df[2:].reset_index(name='count').sort_values(by='count',ascending=False)
    gl = count_df[count_df['count']>0][1:]['index']
    print(d , len(list(gl))) #, gl)

In [11]:
cdf = get_count_df(df, 'OV')
cdf

3469      RYR2
881     COL6A3
3468      RYR1
2994     PKHD1
1243      EGFR
         ...  
34       ABHD2
38      ABI3BP
39        ABL1
54       ACKR1
55       ACKR3
Name: index, Length: 1957, dtype: object

In [15]:
'ID' in list(cdf)

False

In [26]:
def get_count_df(df, d, th):
    count_df = df[df.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum)
    count_df = count_df[2:].reset_index(name='count').sort_values(by='count',ascending=False)
    gl = count_df[count_df['count']>th][1:]['index']
    return gl
from collections import defaultdict
expert_dict = defaultdict()

df_ = data.copy()

df_['SUBCLASS'] = df.loc[:, df.columns[1:2]]
for d in disease_list:
    all_g = set(df[df.SUBCLASS==d].columns)
    gl = get_count_df(df_, d, 1)
    try:
        g_expert = pd.read_csv('./reference_data/'+d+'.csv')
        aa = set(g_expert['g.name'])        
        
    except FileNotFoundError:
        aa = set(get_count_df(df_, d, 5))
        
    expdict = all_g.intersection(aa)
    expert_dict[d] = expdict
    print(d,  len(list(gl)), len(aa), len(expdict))
    


KIPAN 1873 699 307
SARC 513 69 38
SKCM 3533 69 31
KIRC 1050 132 69
GBMLGG 1131 181 117
STES 3650 69 38
BRCA 2625 1004 497
THCA 197 58 34
LIHC 1161 179 104
HNSC 1948 55 37
PAAD 983 628 287
OV 818 317 133
PRAD 370 611 311
UCEC 3641 89 54
LAML 49 1007 493
COAD 2950 557 278
ACC 555 38 23
LGG 288 111 53
LUSC 2399 629 246
LUAD 2116 467 467
CESC 1652 42 22
PCPG 72 3 3
THYM 30 19 10
BLCA 1774 121 70
TGCT 430 23 9
DLBC 553 9 4


In [36]:
#preprocessing for total
#for model csv
total = pd.concat([data,df_test])
f = lambda x : '' if x == 'WT' else str(x)
total = total.map(f)



In [7]:
total

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,R89,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,,,,,,,,,,,...,,,,,,,,,,
2542,,,,,,,,,,,...,,,,,,,,,,
2543,,,,,,L21,P22,R5M,G60,I24,...,S20,L23,,,L30,,N25,G67,,
2544,,,,,,,,,R15,,...,,,,,,,,,,


In [7]:
### ###
nCols = len(data.columns)
from collections import defaultdict
cDic = defaultdict()
for i, c in enumerate(data.columns):
    cDic[i] = c

In [8]:
def make_ndf (data, df_test):
    #data.drop('SUBCLASS', axis=1,inplace=True)
    #target = pd.concat([data,df_test])
    
    #2칸=>1칸
    ndf = pd.DataFrame()
    
    for i in range(nCols//2):
        ndf[cDic[i*2]] = total[total.columns[i*2]] +' '+ total[total.columns[(i*2+1)]]
    return ndf

In [ ]:
#preprocessing
#for writing csv
def preprocessing_for_writing():
    f = lambda x : '' if x == 'WT' else x
    tdf = data.map(f)
    tdf['SUBCLASS'] = df.loc[:, df.columns[1:2]]
    tdf


In [9]:
df.loc[:, df.columns[:2]]

,ID,SUBCLASS
0,TRAIN_0000,KIPAN
1,TRAIN_0001,SARC
2,TRAIN_0002,SKCM
3,TRAIN_0003,KIRC
4,TRAIN_0004,GBMLGG
...,...,...
6196,TRAIN_6196,LUAD
6197,TRAIN_6197,LGG
6198,TRAIN_6198,COAD
6199,TRAIN_6199,TGCT


In [459]:
def getnDF(d):
    #2칸=>1칸
    ndf = pd.DataFrame()
    target = tdf[tdf['SUBCLASS']==d]
    target.drop('SUBCLASS', axis=1,inplace=True)
    for i in range(nCols//2):
        ndf[cDic[i*2]] = target[target.columns[i*2]] +' '+ target[target.columns[(i*2+1)]]
    return ndf

In [467]:
#for writing
for d in tdf['SUBCLASS'].unique():
    ndf = getnDF(d)
    for c in ndf.columns:
        ndf[c] = ndf[c].apply(lambda x : '-' if (x =='' or x==' ') else 'X')
    ndf.to_csv('./data1/bstr-'+d+'-1.csv')


In [308]:
a = [1,2,3,4,5,6,7,8]
for i in range (0,len(a)//2):
    print(i*2, (i+1)*2)

0 2
2 4
4 6
6 8


In [398]:
#2칸=>1칸
ndf = pd.DataFrame()
for i in range(10): #range(nCols//2):
    print(i, i+1) 
    #ndf[cDic[i*2]] =  data[data.columns[ i*2] or data.columns[  (i+1) * 2]]  


0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10


In [403]:
#2칸=>1칸
ndf = pd.DataFrame()
for i in range(10): #range(nCols//2):
    print(i*2, i*2 +1) 
    #ndf[cDic[i*2]] =  data[data.columns[ i*2] or data.columns[  (i+1) * 2]]  


0 1
2 3
4 5
6 7
8 9
10 11
12 13
14 15
16 17
18 19


In [404]:
#2칸=>1칸
ndf = pd.DataFrame()
for i in  range(nCols//2):
    ndf[cDic[i*2]] = data[data.columns[i*2]] +' '+ data[data.columns[(i*2+1)]]
    #print(i*2, (i+1) * 2) 
    #strr = data[data.columns[i*2]] data[data.columns[ (i+1) * 2]]
    #ndf[cDic[i*2]] =  data[data.columns[ i*2] or data.columns[  (i+1) * 2]]  
ndf


,A2M,AADAT,ABAT,ABCA2,ABCA4,ABCA6,ABCA9,ABCB11,ABCB6,ABCB8,...,ZC3H14,ZFP36,ZFPM2,ZMIZ1,ZNF185,ZNF292,ZNF639,ZNFX1,ZPBP,ZWINT
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,R895R,,,,,,,T426T,,,...,,,,,,,,,,
3,,,,,,,,,,,...,Y904H,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,,,,,,,,,,L101M,...,,,,,,,,,,
6197,,,,,,,,,,,...,,,,,,,,,,
6198,,,,,,,,,,,...,,,,,,,,,,T181S
6199,,,,,,,,,,,...,,,,,,,,,,


In [405]:
ndf

,A2M,AADAT,ABAT,ABCA2,ABCA4,ABCA6,ABCA9,ABCB11,ABCB6,ABCB8,...,ZC3H14,ZFP36,ZFPM2,ZMIZ1,ZNF185,ZNF292,ZNF639,ZNFX1,ZPBP,ZWINT
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,R895R,,,,,,,T426T,,,...,,,,,,,,,,
3,,,,,,,,,,,...,Y904H,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,,,,,,,,,,L101M,...,,,,,,,,,,
6197,,,,,,,,,,,...,,,,,,,,,,
6198,,,,,,,,,,,...,,,,,,,,,,T181S
6199,,,,,,,,,,,...,,,,,,,,,,


In [424]:
cDic

defaultdict(None,
            {0: 'A2M',
             1: 'AAAS',
             2: 'AADAT',
             3: 'AARS1',
             4: 'ABAT',
             5: 'ABCA1',
             6: 'ABCA2',
             7: 'ABCA3',
             8: 'ABCA4',
             9: 'ABCA5',
             10: 'ABCA6',
             11: 'ABCA8',
             12: 'ABCA9',
             13: 'ABCB1',
             14: 'ABCB11',
             15: 'ABCB4',
             16: 'ABCB6',
             17: 'ABCB7',
             18: 'ABCB8',
             19: 'ABCB9',
             20: 'ABCC1',
             21: 'ABCC2',
             22: 'ABCC3',
             23: 'ABCC4',
             24: 'ABCC5',
             25: 'ABCC8',
             26: 'ABCD1',
             27: 'ABCD2',
             28: 'ABCD3',
             29: 'ABCE1',
             30: 'ABCF2',
             31: 'ABCG2',
             32: 'ABCG4',
             33: 'ABCG8',
             34: 'ABHD2',
             35: 'ABHD4',
             36: 'ABHD6',
             37: 'ABI1',
        

In [426]:
ndf

,A2M,AADAT,ABAT,ABCA2,ABCA4,ABCA6,ABCA9,ABCB11,ABCB6,ABCB8,...,ZC3H14,ZFP36,ZFPM2,ZMIZ1,ZNF185,ZNF292,ZNF639,ZNFX1,ZPBP,ZWINT
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,R895R,,,,,,,T426T,,,...,,,,,,,,,,
3,,,,,,,,,,,...,Y904H,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,,,,,,,,,,L101M,...,,,,,,,,,,
6197,,,,,,,,,,,...,,,,,,,,,,
6198,,,,,,,,,,,...,,,,,,,,,,T181S
6199,,,,,,,,,,,...,,,,,,,,,,


In [411]:
#df[df.columns[1::2]]
#df[df.columns[0::2]]

In [351]:
df[df.ABCB8 !='WT']


,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
14,TRAIN_0014,SKCM,WT,WT,WT,WT,WT,F157L S1042F,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
354,TRAIN_0354,DLBC,A615A,WT,WT,WT,WT,R1228R P2150P,WT,V150V,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
468,TRAIN_0468,OV,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
489,TRAIN_0489,CESC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
532,TRAIN_0532,LUSC,WT,WT,WT,WT,WT,WT,R2400W A903S,WT,...,E2679*,WT,WT,WT,WT,WT,WT,WT,WT,WT
562,TRAIN_0562,BLCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
717,TRAIN_0717,STES,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,Y46H,WT,WT
843,TRAIN_0843,SKCM,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,S1244N,WT,WT,WT,WT,WT
1138,TRAIN_1138,BRCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1369,TRAIN_1369,SKCM,E1311K E342K R147C E1311K E342K R147C,P307S T163T P307S T163T,P244L P244L,WT,P239S P239S,P91S P91S,S1492F A1262A F1948F S1237F L1683L S1492F A126...,G495G P72L L310L D1089D G495G P72L L310L D1089D,...,P873S L2423V R2173* S1473F P873S L2423V R2173*...,E54K E54K,WT,WT,F380F S197F F380F S197F,G378S L264L P349P G121I I207I G378S L264L P349...,E143K E143K,L489L L489L,WT,WT


In [359]:
df[df.ABCB8 !='WT']['SUBCLASS'].value_counts()

SUBCLASS
STES      10
SKCM       7
LUAD       6
COAD       5
BRCA       5
UCEC       4
CESC       3
BLCA       3
DLBC       2
OV         2
TGCT       2
LUSC       1
ACC        1
GBMLGG     1
KIRC       1
KIPAN      1
PAAD       1
SARC       1
Name: count, dtype: int64

In [314]:
data[data.columns[0*2]]

0          WT
1          WT
2       R895R
3          WT
4          WT
        ...  
6196       WT
6197       WT
6198       WT
6199       WT
6200       WT
Name: A2M, Length: 6201, dtype: object

In [315]:
data[data.columns[2]]

0       WT
1       WT
2       WT
3       WT
4       WT
        ..
6196    WT
6197    WT
6198    WT
6199    WT
6200    WT
Name: AADAT, Length: 6201, dtype: object

In [305]:
data.loc[:, data.columns[2:4]]

,AADAT,AARS1
0,WT,WT
1,WT,WT
2,WT,WT
3,WT,WT
4,WT,WT
...,...,...
6196,WT,WT
6197,WT,WT
6198,WT,WT
6199,WT,WT


In [276]:
df

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,TRAIN_6196,LUAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,TRAIN_6197,LGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,TRAIN_6198,COAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,TRAIN_6199,TGCT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TRAIN_0001,SARC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TRAIN_0002,SKCM,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TRAIN_0003,KIRC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TRAIN_0004,GBMLGG,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,TRAIN_6196,LUAD,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6197,TRAIN_6197,LGG,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6198,TRAIN_6198,COAD,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6199,TRAIN_6199,TGCT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [190]:
data.loc[:, 'A2M']


0       0
1       0
2       1
3       0
4       0
       ..
6196    0
6197    0
6198    0
6199    0
6200    0
Name: A2M, Length: 6201, dtype: int64

In [197]:
data.loc[:, data.columns[0:3]]

,A2M,AAAS,AADAT
0,0,0,0
1,0,0,0
2,1,0,0
3,0,0,0
4,0,0,0
...,...,...,...
6196,0,0,0
6197,0,0,0
6198,0,0,0
6199,0,0,0


In [280]:
df.loc[:, df.columns[2:5]].agg(sum)

A2M      148
AAAS      44
AADAT     25
dtype: object

In [201]:
data.loc[:, data.columns[0:4]].agg(sum)

A2M      148
AAAS      44
AADAT     25
AARS1      0
dtype: int64

In [207]:
data

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [216]:
df[df.SUBCLASS == 'KIPAN']

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
16,TRAIN_0016,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
17,TRAIN_0017,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
59,TRAIN_0059,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
66,TRAIN_0066,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132,TRAIN_6132,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,S196G,WT,WT,WT,WT,WT
6137,TRAIN_6137,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6163,TRAIN_6163,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6186,TRAIN_6186,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [215]:
target.loc[:, data.columns[0:1]].agg(sum) 

A2M    WTWTWTWTWTWTWTWTWTWTWTWTS660RWTWTWTWTWTWTWTWTW...
dtype: object

In [ ]:
ndf = pd.DataFrame()
for i in range(0, 1): #len(glist)):
    if i != 0 or i != len(glist)-1:
        ndf[]
        #ndf[glist[i]] =
        
        
        
    #print(i, glist[i])


array(['KIPAN', 'SARC', 'SKCM', 'KIRC', 'GBMLGG', 'STES', 'BRCA', 'THCA',
       'LIHC', 'HNSC', 'PAAD', 'OV', 'PRAD', 'UCEC', 'LAML', 'COAD',
       'ACC', 'LGG', 'LUSC', 'LUAD', 'CESC', 'PCPG', 'THYM', 'BLCA',
       'TGCT', 'DLBC'], dtype=object)

In [61]:
df[df.SUBCLASS=='ACC']

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
30,TRAIN_0030,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44,TRAIN_0044,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,TRAIN_0050,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183,TRAIN_0183,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
656,TRAIN_0656,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5901,TRAIN_5901,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5917,TRAIN_5917,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6035,TRAIN_6035,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6082,TRAIN_6082,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
df[df.SUBCLASS=='ACC']['PLEC']

30                    A2113A L1321L
44             A2113A A2106A K2047*
50                               WT
183            A2113A A2106A L1321L
656                              WT
                   ...             
5901                             WT
5917    A2113A A2106A A1697A L1321L
6035                         A2507A
6082                         A2495A
6148                             WT
Name: PLEC, Length: 72, dtype: object

In [69]:
da = df[df.SUBCLASS=='LAML']
cnt = []
for c in da.columns[2:]:
        cnt.append(da[c].apply(lambda x : x!=0).agg(sum))
max(cnt)

41

In [62]:

count_df = df[df.SUBCLASS=='LAML'].apply(lambda x : x!=0).agg(sum).sort_values(ascending=False).reset_index(name='count')
print(count_df)
total_rows = count_df[count_df['index']== 'ID']['count'].values[0]
print(count_df[count_df['count'] >= total_rows*0.75][2:])



#print(count_df[count_df['count']]>count_df['ID'])
#gene_list = count_df[count_df['count']>0]['index'].values
#print(len(gene_list))
#gene_list

         index  count
0           ID    158
1     SUBCLASS    158
2         NPM1     41
3         IDH1     18
4         IDH2     17
...        ...    ...
4381     ZNRF4      0
4382       A2M      0
4383      ZW10      0
4384     ZWINT      0
4385       ZYX      0

[4386 rows x 2 columns]
Empty DataFrame
Columns: [index, count]
Index: []


In [50]:
df[df.SUBCLASS=='SKCM'].apply(lambda x : x!='WT').columns


Index(['ID', 'SUBCLASS', 'A2M', 'AAAS', 'AADAT', 'AARS1', 'ABAT', 'ABCA1',
       'ABCA2', 'ABCA3',
       ...
       'ZNF292', 'ZNF365', 'ZNF639', 'ZNF707', 'ZNFX1', 'ZNRF4', 'ZPBP',
       'ZW10', 'ZWINT', 'ZYX'],
      dtype='object', length=4386)

In [74]:
def get_gene_list2():
    genes = []
    gene_set = set()
    for disease in df["SUBCLASS"].unique():
        disease_df = df[df["SUBCLASS"] == disease]

        non_wt_counts = (disease_df != 'WT').sum()  # "WT"가 아닌 행 개수
        non_wt_counts_sorted = non_wt_counts.sort_values(ascending=False) #빈도 수가 높은 순서대로 정렬

        n = 20 # 추출한 상위 유전자 개수
        genes.extend(list(non_wt_counts_sorted[2:2+n].index))

        #print(f'{disease} - 전체 행의 개수 : {len(disease_df)}')
        dd = non_wt_counts_sorted[2:n].reset_index()
        gene_list1 = dd['index'].values
        gene_set.update(gene_list1)
        #print(gene_list1)
    return gene_set

In [8]:
data

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [10]:
df.loc[:, df.columns[1:2]]

,SUBCLASS
0,KIPAN
1,SARC
2,SKCM
3,KIRC
4,GBMLGG
...,...
6196,LUAD
6197,LGG
6198,COAD
6199,TGCT


In [14]:
df_ = data.copy()
df_['SUBCLASS']= df.loc[:, df.columns[1:2]]
df_
    

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX,SUBCLASS
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,KIPAN
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,SARC
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,SKCM
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,KIRC
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,GBMLGG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,LUAD
6197,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,LGG
6198,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,T181S,WT,COAD
6199,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,TGCT


In [6]:
#### ####

In [22]:
diseaseList = df.SUBCLASS.value_counts().rename_axis('disease').reset_index(name='counts')
diseaseList

,disease,counts
0,BRCA,786
1,KIPAN,515
2,GBMLGG,461
3,STES,379
4,KIRC,334
5,THCA,324
6,SKCM,276
7,PRAD,266
8,OV,253
9,LGG,229


In [23]:
df[df.SUBCLASS == 'KIPAN'].shape[0]

515

In [12]:
data

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [11]:
selected_genes = set()
for d in disease_list:
    df_ = data.copy()
    df_['SUBCLASS'] = df.loc[:, df.columns[1:2]]
    count_df = df_[df_.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum)
    count_df = count_df[2:].reset_index(name='count')        
    #num_transaction = diseaseList[diseaseList.disease == d]['counts'].values[0]
    th = 5 #if num_transaction > 250 else 5
    #th = num_transaction//divider
    #print( num_transaction, th)
    
    gene_list = count_df[count_df['count']>th]['index'].values
    print(d, len(gene_list))
    selected_genes.update(gene_list)
print(len(selected_genes))

    

KIPAN 287
SARC 32
SKCM 2112
KIRC 69
GBMLGG 148
STES 2033
BRCA 691
THCA 5
LIHC 107
HNSC 334
PAAD 23
OV 48
PRAD 17
UCEC 2006
LAML 10
COAD 936
ACC 52
LGG 17
LUSC 587
LUAD 468
CESC 176
PCPG 4
THYM 2
BLCA 209
TGCT 15
DLBC 18
2924


In [29]:
if 'SUBCLASS' in selected_genes:
    print('ry')

In [12]:
selected_genes.remove("SUBCLASS")

In [13]:
total

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,R89,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,,,,,,,,,,,...,,,,,,,,,,
2542,,,,,,,,,,,...,,,,,,,,,,
2543,,,,,,L21,P22,R5M,G60,I24,...,S20,L23,,,L30,,N25,G67,,
2544,,,,,,,,,R15,,...,,,,,,,,,,


In [28]:
#total = pd.concat([data,df_test] ) #make_ndf(data, df_test)
data = total[:6201]
total

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2542,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2543,WT,WT,WT,WT,WT,L217I,P221P P251P,R5M,G606D,I248Nfs,...,S2049Vfs S1909Vfs,L232R,WT,WT,L305L,WT,N252I N251I,G679V,WT,WT
2544,WT,WT,WT,WT,WT,WT,WT,WT,R1517H,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [14]:
total = total.loc[:, list(selected_genes)]
total

,TRIB2,FOXC2,SLC25A23,CCNA1,EPHB3,CSF3,MEST,NEGR1,LAMB3,ABCB8,...,TLE4,MPP1,CALCR,AIFM1,MFSD6,TLE1,FOSL2,NOTCH4,JAG1,FKBP5
0,,N23,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,S62,,...,,,,,,,,,,
3,,,,,,,,,P12,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,R42,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,,,,,P8P,,,,,,...,,,,,,,,,,
2542,,,,,,,,,,,...,,,,,,,,,,
2543,,,,,A44,L15,,,,R53,...,W58,,,,,P16,,D59,P73,
2544,,,,,,,,,,,...,,,,,,,,,,


In [16]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for c in total.columns:
    total[c] = encoder.fit_transform(total[c])
#total[total.select_dtypes(include=['object']).columns] = df.select_dtypes(include=['object']).apply(lambda col: encoder.fit_transform(col))




In [17]:
df = total[:6201]
df_test = total[6201:]



In [18]:
df

,TRIB2,FOXC2,SLC25A23,CCNA1,EPHB3,CSF3,MEST,NEGR1,LAMB3,ABCB8,...,TLE4,MPP1,CALCR,AIFM1,MFSD6,TLE1,FOSL2,NOTCH4,JAG1,FKBP5
0,0,37,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,127,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,76,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,75,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,0,0,0,0,0,0,0,0,0,30,...,0,0,0,0,0,0,0,0,0,0
6197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df_test

,TRIB2,FOXC2,SLC25A23,CCNA1,EPHB3,CSF3,MEST,NEGR1,LAMB3,ABCB8,...,TLE4,MPP1,CALCR,AIFM1,MFSD6,TLE1,FOSL2,NOTCH4,JAG1,FKBP5
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,33,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,18,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,0,0,0,0,76,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2543,0,0,0,0,8,10,0,0,0,59,...,120,0,0,0,0,60,0,43,90,0
2544,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
from sklearn.preprocessing import LabelEncoder
X = df
y = df_target
encoder2 = LabelEncoder()
y = encoder2.fit_transform(y)
y


array([ 8, 19, 20, ...,  4, 22, 20])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=156)#shuffle=False,stratify = y, 
print(X_train.shape, X_test.shape)

(4340, 2923) (1861, 2923)


In [22]:
dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)


#Multi-class classification

In [23]:
params = {
    'tree_method': 'approx',
    'objective': 'multi:softprob',
    'num_class': 26
}
num_boost_round = 50

dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)
model = xgb.train(params=params, dtrain=dtrain, num_boost_round=num_boost_round , early_stopping_rounds=5, 
                  evals=[(dtrain, 'train'), (dtest, 'valid')],
                  verbose_eval=2)

[0]	train-mlogloss:2.60579	valid-mlogloss:2.70142
[2]	train-mlogloss:2.21000	valid-mlogloss:2.44574
[4]	train-mlogloss:1.97859	valid-mlogloss:2.32603
[6]	train-mlogloss:1.82044	valid-mlogloss:2.25446
[8]	train-mlogloss:1.69942	valid-mlogloss:2.21104
[10]	train-mlogloss:1.60382	valid-mlogloss:2.18132
[12]	train-mlogloss:1.52782	valid-mlogloss:2.15862
[14]	train-mlogloss:1.46402	valid-mlogloss:2.14455
[16]	train-mlogloss:1.41028	valid-mlogloss:2.13522
[18]	train-mlogloss:1.36348	valid-mlogloss:2.12697
[20]	train-mlogloss:1.32234	valid-mlogloss:2.12518
[22]	train-mlogloss:1.28512	valid-mlogloss:2.12306
[24]	train-mlogloss:1.25173	valid-mlogloss:2.12017
[26]	train-mlogloss:1.22031	valid-mlogloss:2.11900
[28]	train-mlogloss:1.19121	valid-mlogloss:2.11754
[30]	train-mlogloss:1.16500	valid-mlogloss:2.12040
[32]	train-mlogloss:1.13955	valid-mlogloss:2.11786
[33]	train-mlogloss:1.12718	valid-mlogloss:2.11969


In [24]:
params = {
    'tree_method': 'approx',
    'objective': 'multi:softprob',
    'learning_rate': 0.05
}
num_boost_round = 200

clf = xgb.XGBClassifier(n_estimators=num_boost_round, early_stopping_rounds=5, **params) 
clf.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test,y_test)],
        verbose=2);

[0]	validation_0-mlogloss:3.13733	validation_1-mlogloss:3.15336
[2]	validation_0-mlogloss:2.95628	validation_1-mlogloss:3.00115
[4]	validation_0-mlogloss:2.81826	validation_1-mlogloss:2.89001
[6]	validation_0-mlogloss:2.70562	validation_1-mlogloss:2.80137
[8]	validation_0-mlogloss:2.61027	validation_1-mlogloss:2.72977
[10]	validation_0-mlogloss:2.52781	validation_1-mlogloss:2.67014
[12]	validation_0-mlogloss:2.45410	validation_1-mlogloss:2.61785
[14]	validation_0-mlogloss:2.38835	validation_1-mlogloss:2.57363
[16]	validation_0-mlogloss:2.32881	validation_1-mlogloss:2.53437
[18]	validation_0-mlogloss:2.27495	validation_1-mlogloss:2.50044
[20]	validation_0-mlogloss:2.22488	validation_1-mlogloss:2.46883
[22]	validation_0-mlogloss:2.17854	validation_1-mlogloss:2.44084
[24]	validation_0-mlogloss:2.13606	validation_1-mlogloss:2.41560
[26]	validation_0-mlogloss:2.09667	validation_1-mlogloss:2.39329
[28]	validation_0-mlogloss:2.05923	validation_1-mlogloss:2.37302
[30]	validation_0-mlogloss:2.0

In [25]:
y_true = y_test
y_pred = clf.predict(X_test)
y_score = clf.predict_proba(X_test)
y_true.shape, y_pred.shape, y_score.shape

((1861,), (1861,), (1861, 26))

In [52]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

0.34712520150456744

In [27]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

0.3444384739387426

In [28]:
print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.70      0.74        23
           1       0.38      0.14      0.21        35
           2       0.27      0.65      0.38       238
           3       0.10      0.03      0.04        39
           4       0.62      0.65      0.64        63
           5       1.00      0.12      0.22        16
           6       0.34      0.31      0.33       150
           7       0.26      0.23      0.25        60
           8       0.27      0.36      0.31       159
           9       0.06      0.04      0.05       104
          10       0.93      0.29      0.44        48
          11       0.27      0.42      0.33        57
          12       0.31      0.17      0.22        48
          13       0.25      0.07      0.11        59
          14       0.41      0.18      0.25        49
          15       0.31      0.49      0.38        93
          16       0.07      0.04      0.05        28
          17       0.18    

In [29]:
metrics.roc_auc_score(y_true, y_score, average='weighted', multi_class='ovr')

0.8541877349290403

In [54]:
clf.predict(df_test)

array([21, 25,  2, ..., 25, 10,  6])

In [30]:
res = clf.predict(df_test)



In [31]:
res = encoder2.inverse_transform(res)

In [32]:

df_test1 = pd.read_csv('test.csv')
res_df = pd.DataFrame ({'ID' : df_test1['ID'], 'SUBCLASS' : res})
res_df


,ID,SUBCLASS
0,TEST_0000,STES
1,TEST_0001,UCEC
2,TEST_0002,THCA
3,TEST_0003,LGG
4,TEST_0004,STES
...,...,...
2541,TEST_2541,COAD
2542,TEST_2542,HNSC
2543,TEST_2543,SKCM
2544,TEST_2544,LAML


In [33]:
res_df.SUBCLASS.value_counts()

SUBCLASS
BRCA      618
COAD      537
KIPAN     218
STES      142
OV        141
GBMLGG    139
UCEC       92
SKCM       83
THCA       79
LGG        71
HNSC       67
LIHC       51
KIRC       46
TGCT       37
LAML       36
PRAD       31
CESC       30
BLCA       23
LUAD       22
LUSC       19
SARC       15
PAAD       14
ACC        13
PCPG       12
THYM        7
DLBC        3
Name: count, dtype: int64

In [34]:
res_df.to_csv('v3.csv',index = False)

In [97]:
pred_probs = xgb_model.predict(dtest)


NameError: name 'xgb_model' is not defined

In [ ]:
import numpy as np
print(np.round(pred_probs[:10], 3))


In [58]:
preds= [round(x) if x > 0 else 0 for x in pred_probs]

In [ ]:
preds

In [60]:
for p in preds:
    if p<0:
        print(p)


In [ ]:
encoder.inverse_transform(preds)

In [35]:
model = RandomForestClassifier(
            n_estimators=50, ## 붓스트랩 샘플 개수 또는 base_estimator 개수
            criterion='entropy', ## 불순도 측도
            max_depth=65, ## 개별 나무의 최대 깊이
            #max_features='sqrt', ## 매 분리시 랜덤으로 뽑을 변수 개수
            #max_samples=1.0, ## 붓스트랩 샘플 비율 => 1이면 학습데이터를 모두 샘플링한다.
            #bootstrap=True, ## 복원 추출,  False이면 비복원 추출
            oob_score=True, ## Out-of-bag 데이터를 이용한 성능 계산
            random_state=100
).fit(X,y)


In [ ]:
## 예측
print(model.predict(X)[:3]) 
print()


In [ ]:
X[:3]

In [ ]:
## 성능 평가
print(model.oob_score_) ## Out-of-bag 성능 평가 점수
print('정확도 : ', model.score(X,y)) ## 테스트 성능 평가 점수(Accuracy)
print()
## 변수 중요도
print(model.feature_importances_)




In [ ]:
## 성능 평가
print(model.oob_score_) ## Out-of-bag 성능 평가 점수
print('정확도 : ', model.score(X,y)) ## 테스트 성능 평가 점수(Accuracy)
print()
## 변수 중요도
print(model.feature_importances_)




In [ ]:
res = model.predict(df_test)
res

In [ ]:
res = model.predict(df_test)
res

In [ ]:
df_test1 = pd.read_csv('test.csv')
res_df = pd.DataFrame ({'ID' : df_test1['ID'], 'SUBCLASS' : res})
res_df



In [24]:
res_df.to_csv('res04.csv',index = False)

In [ ]:
df[df.SUBCLASS == 'BRCA'].sum().sort_values(ascending=False).str.replace('WT','')

In [ ]:
df[df['SUBCLASS']=='DLBC']

In [ ]:
import numpy as np 
dfs = df[df['SUBCLASS']=='DLBC']
dfs.columns

In [ ]:
for c in dfs.columns:
    dfs[c] = np.where(dfs[c] == 'WT', 0, 1)
dfs

    

In [ ]:
dfs = df[df['SUBCLASS']=='DLBC']
dfs.apply(lambda x: x(if x=='WT' else x=1))

In [ ]:
for  x in df.iloc[0]:
    if x != 'WT': print(x)


In [ ]:
len(df_test.columns)

In [ ]:
len(df_test)

In [37]:
a1 = ['DDX10', 'ELOVL5']
a2 = ['AAAS', 'ABCB4', 'ABCC1', 'ABCC4', 'ABL1', 'ACKR3', 'ACOX2', 'ACSL1', 'ACSS2', 'ACVR1B', 'ADAD1', 'ADAM10', 'ADAM2', 'ADAMTS5', 'ADCY2', 'ADH5', 'ADIPOQ', 'AGO4', 'AHNAK', 'AIFM3', 'AKAP12', 'ALAS1', 'ALDH1A1', 'ALDH2', 'ALS2', 'ANLN', 'ARHGEF7', 'ATAD2', 'ATP1A1', 'ATP7A', 'ATXN1', 'AUTS2', 'AVPR1A', 'AXIN2', 'BAZ2A', 'BIRC2', 'BRAF', 'BTBD3', 'CACNA1A', 'CACNA1H', 'CAMKK2', 'CASP8', 'CAST', 'CCR5', 'CCT6A', 'CD302', 'CD47', 'CD7', 'CDADC1', 'CDH13', 'CDH16', 'CDH3', 'CDH4', 'CDK5RAP2', 'CDON', 'CELSR1', 'CELSR2', 'CENPJ', 'CKAP4', 'CLDN9', 'COL11A1', 'COL16A1', 'COL2A1', 'COL5A1', 'CORO1A', 'CPT1A', 'CREBBP', 'CSTF3', 'CTSB', 'CTSV', 'CTTN', 'CXCR3', 'CYP2C18', 'DAAM1', 'DDX10', 'DGCR8', 'DHRS2', 'DIO1', 'DLC1', 'DLK2', 'DNM1L', 'DOCK10', 'DOCK2', 'DSC3', 'DST', 'DUSP6', 'DVL2', 'DYRK1A', 'DYRK3', 'EIF3A', 'EIF4G3', 'ELL', 'ELOVL5', 'ELOVL6', 'ENO1', 'ERN1', 'ESR1', 'EVL', 'EWSR1', 'EXO1', 'EZR', 'F12', 'F5', 'FADS2', 'FBXO9', 'FBXW7', 'FCN1', 'FEN1', 'FGD6', 'FKBP4', 'FLNA', 'FLNB', 'FN1', 'FSCN1', 'FXN', 'GEMIN4', 'GJB3', 'GMPR2', 'GPD1', 'GPI', 'GPRC5C', 'GSS', 'HSPB1', 'HUWE1', 'ICAM5', 'IDH2', 'IDH3G', 'IGF1R', 'IL17RA', 'IL1RL1', 'IL6', 'IPO4', 'IRF9', 'IRS4', 'ISG20L2', 'ISL1', 'ITIH5', 'ITPK1', 'JAG2', 'JUP', 'KALRN', 'KCND1', 'KDM3A', 'KIF3C', 'KIF5B', 'KIFAP3', 'KLF10', 'KMT2D', 'KRT15', 'KRT31', 'LAMA1', 'LAMA3', 'LAMB3', 'LGALS3BP', 'LIG1', 'LRP1', 'LTBP1', 'LTF', 'LUM', 'MAP3K1', 'MAP4K1', 'MAPKAP1', 'MAST3', 'MCCC1', 'MEF2D', 'MEIS2', 'MLF1', 'MMP15', 'MMP16', 'MMP2', 'MMP9', 'MPI', 'MVK', 'MYH10', 'MYH3', 'MYH9', 'MYOT', 'NAV2', 'NCOA3', 'NCSTN', 'NDST2', 'NDUFAF4', 'NET1', 'NINJ1', 'NKX3-1', 'NMI', 'NOD1', 'NOLC1', 'NOS2', 'NOTCH2', 'NTRK3', 'NUFIP1', 'NXF1', 'ODF2', 'OLFM1', 'P2RY1', 'PAIP1', 'PCDHB1', 'PCM1', 'PEG3', 'PEX12', 'PFKL', 'PHF3', 'PHGDH', 'PHKA2', 'PIK3CD', 'PKHD1', 'PLAGL1', 'PLEC', 'PLXNB1', 'PPARGC1A', 'PRKCB', 'PTCH1', 'PTPN6', 'PTPRC', 'PTPRG', 'PTPRM', 'PYGL', 'RET', 'RHOT1', 'ROCK1', 'ROS1', 'RPL3L', 'RXRG', 'RYR1', 'RYR2', 'SCARB1', 'SCN9A', 'SCUBE2', 'SEC14L1', 'SEC24D', 'SFRP1', 'SHROOM2', 'SIDT1', 'SLC12A3', 'SLC35A3', 'SLC39A6', 'SLC6A12', 'SLC6A9', 'SLC7A1', 'SLN', 'SMAD1', 'SMC1A', 'SMC3', 'SMC4', 'SMOX', 'SMTN', 'SNAPC4', 'SPTAN1', 'SREBF2', 'ST14', 'STAB1', 'STAG3', 'STARD3', 'STAU1', 'SVIL', 'SYNJ2', 'TAOK2', 'TCF7L2', 'TDO2', 'TEAD4', 'TFDP1', 'TG', 'THBD', 'THBS3', 'TIMELESS', 'TIMM10', 'TJP3', 'TK1', 'TLE4', 'TM7SF3', 'TMEM164', 'TNC', 'TNFRSF1A', 'TNIP1', 'TNIP2', 'TOP1', 'TRIB2', 'TSC1', 'TSPYL2', 'TUBA3C', 'TXNRD2', 'TYK2', 'UBE2D1', 'UBE2L6', 'UNC13B', 'UNC5C', 'UQCRB', 'USO1', 'WEE1', 'WNT1', 'WWC1', 'XPC', 'YWHAZ', 'ZNF292', 'ZNFX1']
set(a1).intersection(set(a2))

{'DDX10', 'ELOVL5'}

In [39]:
a1[0] in a2

True

In [44]:
df.shape[0]

6201

In [75]:
df.iloc[4984:4985].iloc[:, 2:].replace('WT', '').

A2M           
AAAS     R194Q
AADAT         
AARS1         
ABAT          
         ...  
ZNRF4         
ZPBP          
ZW10          
ZWINT         
ZYX      A525T
Length: 4384, dtype: object

In [78]:
to_compare = ['DDX10', 'ELOVL5']
for i in range(0, df.shape[0]): 
    
    bset = set()
    
    row = df.iloc[i:i+1].iloc[:, 2:]#remove ID,SUBCLASS
    count_df = row.apply(lambda x : x!='WT').agg(sum).reset_index(name='count')
    g_setl = count_df.loc[count_df['count'] > 0]['index']
    
    if to_compare[0] in set(g_setl):
        print(i)
        print( set(to_compare).intersection(set(g_setl)) )
        for a in to_compare: 
            bset.add(row[a].values[0])
        print(bset)
        print()
    

244
{'DDX10'}
{'WT', 'E186Q'}

286
{'DDX10'}
{'WT', 'Y292D'}

379
{'DDX10'}
{'WT', 'L128L'}

406
{'DDX10'}
{'WT', 'E445* E466* E750D'}

562
{'DDX10'}
{'WT', 'P436L'}

601
{'DDX10'}
{'WT', 'E580*'}

680
{'DDX10'}
{'WT', 'M825T'}

1120
{'DDX10'}
{'WT', 'E658*'}

1202
{'DDX10', 'ELOVL5'}
{'Q622H', 'S149S'}

1233
{'DDX10'}
{'WT', 'E712D'}

1476
{'DDX10'}
{'K119R', 'WT'}

1489
{'DDX10'}
{'WT', 'Q704H'}

1789
{'DDX10'}
{'WT', 'Q462P Y551* K665R'}

1883
{'DDX10'}
{'P421L', 'WT'}

1924
{'DDX10'}
{'WT', 'A613T'}

2133
{'DDX10'}
{'WT', 'D808D'}

2189
{'DDX10'}
{'WT', 'E591A'}

2377
{'DDX10'}
{'WT', 'M825T'}

2463
{'DDX10'}
{'WT', 'S171* E864K'}

2484
{'DDX10'}
{'S302G', 'WT'}

2690
{'DDX10'}
{'WT', 'A113T'}

2707
{'DDX10'}
{'WT', 'S474C'}

2713
{'DDX10'}
{'WT', 'K410K'}

2732
{'DDX10'}
{'WT', 'K737E'}

3246
{'DDX10'}
{'WT', 'R265H'}

3528
{'DDX10'}
{'D849Y', 'WT'}

3666
{'DDX10'}
{'WT', 'T819I'}

4081
{'DDX10'}
{'WT', 'K78R R309I E547*'}

4210
{'DDX10'}
{'WT', 'T621T'}

4228
{'DDX10'}
{'WT', 'A1

In [ ]:

1202
{'DDX10', 'ELOVL5'}
{'Q622H', 'S149S'}


4656
{'DDX10', 'ELOVL5'}
{'R100H', 'S117S'}

4984
{'DDX10', 'ELOVL5'}
{'E123*', 'V450A D588N E618*'}

5049
{'DDX10', 'ELOVL5'}
{'T140M', 'D215N'}


In [ ]:
G503V L179R

In [79]:
df.iloc[1202]

ID           TRAIN_1202
SUBCLASS           UCEC
A2M               L311P
AAAS                 WT
AADAT                WT
               ...     
ZNRF4                WT
ZPBP        V219I R115H
ZW10                 WT
ZWINT                WT
ZYX               P260T
Name: 1202, Length: 4386, dtype: object

In [80]:
df.iloc[4656]

ID          TRAIN_4656
SUBCLASS          COAD
A2M                 WT
AAAS             P516L
AADAT               WT
               ...    
ZNRF4               WT
ZPBP                WT
ZW10                WT
ZWINT               WT
ZYX                 WT
Name: 4656, Length: 4386, dtype: object

In [81]:
df.iloc[4984]

ID          TRAIN_4984
SUBCLASS          UCEC
A2M                 WT
AAAS             R194Q
AADAT               WT
               ...    
ZNRF4               WT
ZPBP                WT
ZW10                WT
ZWINT               WT
ZYX              A525T
Name: 4984, Length: 4386, dtype: object

In [82]:
df.iloc[5049]

ID           TRAIN_5049
SUBCLASS           UCEC
A2M         K271N N159Y
AAAS                 WT
AADAT                WT
               ...     
ZNRF4                WT
ZPBP                 WT
ZW10              S161Y
ZWINT                WT
ZYX                  WT
Name: 5049, Length: 4386, dtype: object

In [56]:
to_compare[1]

'ELOVL5'